In [1]:
import py2neo
import pandas as pd
import numpy as np
import math

In [2]:
from py2neo import Graph, Node, Relationship, NodeMatcher

In [3]:
def is_not_nan(value):
    if type(value) != str:
        if math.isnan(value):
            return False
        else:
            return True
    else:
        return True

In [5]:
graph = Graph("bolt://localhost:7687", auth=("neo4j", "password"))

In [6]:
graph.run("MATCH (n) DETACH DELETE n")

(No data)

In [7]:
students = pd.read_csv("McSQL Data - students.csv")
classes = pd.read_csv("McSQL Data - classes.csv")
countries = pd.read_csv("McSQL Data - countries.csv")
xtras = pd.read_csv("McSQL Data - extracurriculars.csv")

In [8]:
for i in range(len(students)):
    row = students.iloc[i]
    graph.run("CREATE (:STUDENT {student_id:'" + row['student_id'] + "', first_name:'" + row['first_name'] + "', last_name:'" + row['last_name'] + "', fav_class:'" + row['fav_class'] 
          + "', xtra_id_1:'" + str(row['xtra_id_1']) + "', xtra_id_2:'" + str(row['xtra_id_2']) + "', xtra_id_3:'" + str(row['xtra_id_3']) + "', fav_vacation_id:'" + 
          row['fav_vacation_id'] + "', dream_vacation_id:'" + row['dream_vacation_id'] + "', last_name:'" + row['last_name'] + "'})")

In [9]:
for i in range(len(classes)):
    row = classes.iloc[i]
    graph.run("CREATE (:CLASS {class_id:'" + row['class_id'] + "', class_name:'" + row['class_name'] + "', professor_lname:'" + row['professor_lname'] + "', professor_fname:'" 
          + row['professor_fname'] + "', prereqs_1:'" + str(row['prereqs_1']) + "', prereqs_2:'" + str(row['prereqs_2']) + "', credits:'" + str(row['credits']) + "', gpa:'" 
          + str(row['gpa']) + "'})")

In [10]:
for i in range(len(countries)):
    row = countries.iloc[i]
    graph.run("CREATE (:COUNTRY {country_id:'" + row['country_id'] + "', name:'" + row['name'] + "', capital:'" + row['capital'] + "', population:'" + str(row['population'])
              + "', gdp_per_capita:'" + str(row['gdp_per_capita']) + "', timezone:'" + row['timezone'] + "'})")

In [11]:
for i in range(len(xtras)):
    row = xtras.iloc[i]
    graph.run("CREATE (:EXTRACURRICULAR {xtra_id:'" + str(row['xtra_id']) + "', name:'" + row['name'] + "', semester:'" + row['semester'] + "', director_fname:'" 
              + row['director_fname'] + "', director_lname:'" + row['director_lname'] + "', application:'" + str(row['application']) + "'})")

In [12]:
for i in range(len(students)):
    row = students.iloc[i]
    graph.run("MATCH (s:STUDENT {student_id:'" + row['student_id'] +"'}), (c:CLASS {class_id:'" + row['fav_class'] + "'}) CREATE (s)-[:LIKES]->(c)")

In [13]:
for i in range(len(students)):
    row = students.iloc[i]
    graph.run("MATCH (s:STUDENT {student_id:'" + row['student_id'] +"'}), (c:COUNTRY {country_id:'" + row['fav_vacation_id'] + "'}) CREATE (s)-[:HAS_BEEN_TO]->(c)")

In [14]:
for i in range(len(students)):
    row = students.iloc[i]
    graph.run("MATCH (s:STUDENT {student_id:'" + row['student_id'] +"'}), (c:COUNTRY {country_id:'" + row['dream_vacation_id'] + "'}) CREATE (s)-[:WANTS_TO_GO_TO]->(c)")

In [15]:
for i in range(len(students)):
    row = students.iloc[i]
    graph.run("MATCH (s:STUDENT {student_id:'" + row['student_id'] +"'}), (e:EXTRACURRICULAR {xtra_id:'" + str(row['xtra_id_1']) + "'}) CREATE (s)-[:PARTICIPATES_IN]->(e)")
    graph.run("MATCH (s:STUDENT {student_id:'" + row['student_id'] +"'}), (e:EXTRACURRICULAR {xtra_id:'" + str(row['xtra_id_2']) + "'}) CREATE (s)-[:PARTICIPATES_IN]->(e)")
    if is_not_nan(row['xtra_id_3']):
        temp = int(row['xtra_id_3'])
        graph.run("MATCH (s:STUDENT {student_id:'" + row['student_id'] +"'}), (e:EXTRACURRICULAR {xtra_id:'" + str(temp) + "'}) CREATE (s)-[:PARTICIPATES_IN]->(e)")

In [16]:
for i in range(len(classes)):
    row = classes.iloc[i]
    if is_not_nan(row['prereqs_1']):
        graph.run("MATCH (c1:CLASS {class_id:'" + row['class_id'] +"'}), (c2:CLASS {class_id:'" + row['prereqs_1'] + "'}) CREATE (c1)<-[:IS_PREREQ_OF]-(c2)")
    if is_not_nan(row['prereqs_2']):
        graph.run("MATCH (c1:CLASS {class_id:'" + row['class_id'] +"'}), (c2:CLASS {class_id:'" + row['prereqs_2'] + "'}) CREATE (c1)<-[:IS_PREREQ_OF]-(c2)")